In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F


model_name = "flax-community/papuGaPT2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cpu")

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label



def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to("cpu")
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()


def beam_search(variants, beam_width=3):
    # beam search inicjalizuje beam jako pustą listę
    beam = [(0, "")]  # każdy element to (prawdopodobieństwo, sekwencja)

    for word_variants in variants:
        new_beam = []

        # przeglądanie każdej ścieżki w beam
        for prob, seq in beam:
            for variant in word_variants:
                # tworzenie nowego tekstu na podstawie wariantu
                new_seq = f"{seq} {variant}".strip()
                new_prob = sentence_prob(new_seq)

                # dodajemy nową ścieżkę do beam
                new_beam.append((new_prob, new_seq))


        beam = sorted(new_beam, key=lambda x: x[0], reverse=True)[:beam_width]

    return beam[0][1]

def main(input_string):
    words_variants = [group.split('|') for group in input_string.split() if group]
    best_sequence = beam_search(words_variants)
    return best_sequence


input_string = """\
wprost|wyprosty|wyprostu|wyprost uwielbiała|wielbił|wielbiła|uwielbił|wielbiło|uwielbiał|uwielbiało|uwielbiały
słuchać|osłuchać|słychać|usłuchać o|i|e|a|ó|ę|y|ą|u
wartościach własnych|owłosionych macierzy|mocarz|macierzą|macierze|mocarza|mocarze|mocarzy|macierz
"""


output = main(input_string)
print(output)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


wprost uwielbiał słuchać o wartościach własnych macierzy


In [ ]:
test="Słońce|Końce|Mące świeciło|świergotało|świeżbiło wysoko|wysoki|wyskoki na niebie|niedzielę|niewiele, kiedy|kły|kłody zobaczyliśmy|zboczyliśmy|zmoczyliśmy domek|bobek|tomek na horyzoncie|froncie|koncie"
print(main(test))

Słońce świeciło wysoko na niebie kiedy zobaczyliśmy domek na horyzoncie


In [ ]:
test = "Słonko|Słońce|Słowik świeciło|śpiewało|świszczało nad stawem|statkiem|strachem, gdy|gładko|gniewnie zauważyliśmy|zanotowaliśmy|zasłyszeliśmy że|ze|z zaczyna|zmoczyła|zawita się|śpię|szły robić|rozbić|rodzić ciemno|cienko|ciężko"
print(main(test))


Słońce świeciło nad stawem gdy zauważyliśmy że zaczyna się robić ciemno
